In [1]:
%cd "/content/drive/MyDrive/Tenx program/week-5"

/content/drive/MyDrive/Tenx program/week-5


# Feature Engineering

In [16]:
import sys, os

In [8]:
!python -m src.data_processing \
       --raw data/raw/data.csv \
       --out data/processed/features.parquet

✓ Saved processed features → /content/drive/MyDrive/Tenx program/week-5/data/processed/features.parquet


In [3]:
!pip install -r requirements.txt

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.7/24.7 MB 75.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 76.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.9/57.9 kB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 242.7/242.7 kB 18.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 147.8/147.8 kB 12.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.9/114.9 kB 9.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.0/85.0 kB 7.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.6/63.6 kB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 733.8/733.8 kB 37.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 203.4/203.4 kB 14.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.8/65.8 kB 5.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 118.5/118.5 kB 8.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 196.

In [9]:
!pytest -q
!pytest tests/test_data_processing.py -q

..                                                                       [100%]
2 passed in 2.43s
..                                                                       [100%]
2 passed in 1.46s


# target engineering

In [17]:
if os.getcwd() not in sys.path:
    sys.path.insert(0, os.getcwd())

!python -m src.target_engineering \
        --raw data/raw/data.csv \
        --out data/processed/high_risk_labels.parquet \
        --seed 42

✓ Saved high-risk labels → /content/drive/MyDrive/Tenx program/week-5/data/processed/high_risk_labels.parquet


In [18]:
if os.getcwd() not in sys.path:
    sys.path.insert(0, os.getcwd())

!pytest tests/test_target_engineering.py -q

.                                                                        [100%]
1 passed in 1.47s


# Model Training and Tracking

In [26]:
if os.getcwd() not in sys.path:
    sys.path.insert(0, os.getcwd())

# ONE-OFF: build the merged training file
import pandas as pd
feat = pd.read_parquet("data/processed/features.parquet")
lab  = pd.read_parquet("data/processed/high_risk_labels.parquet")
feat["is_high_risk"] = lab["is_high_risk"].values
feat.to_parquet("data/processed/features_with_target.parquet", index=False)

# Train!
!python -m src.train \
        --features data/processed/features_with_target.parquet \
        --model-name credit_scoring_model \
        --test-size 0.25 \
        --seed 42

2025/06/30 06:15:52 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.
2025/06/30 06:16:19 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.
✓ Registered credit_scoring_model v2 as alias 'champion'  (ROC-AUC=1.000)


In [31]:
!pytest tests/test_train_utils.py -q

.                                                                        [100%]
1 passed in 3.75s
